In [0]:
df_dep_a = spark.read.parquet("/mnt/bronze/hosa/departments")
df_dep_b = spark.read.parquet("/mnt/bronze/hosb/departments")

In [0]:
df_dep_a.show()

In [0]:
from pyspark.sql.functions import col,lit
from pyspark.sql.functions import concat

df_mearged = df_dep_a.unionByName(df_dep_b)

# Create dept_id column and rename deptid to src_dept_id
df_mearged = df_mearged.withColumn("SRC_Dept_id", col("deptid"))\
    .withColumn("Dept_id",concat(col("deptid"),lit("-"),col("datasource")))\
        .drop("deptid")

display(df_mearged)

In [0]:
df_mearged.createOrReplaceTempView("departments")


In [0]:

%sql
CREATE TABLE IF NOT EXISTS silver.departments (
    Dept_id string,
    SRC_Dept_id string,
    Name string,
    datasource string,
    is_bad_data boolean
)
USING DELTA;

TRUNCATE TABLE silver.departments;


In [0]:
%sql

INSERT INTO silver.departments
SELECT 
Dept_id,
SRC_Dept_id ,
Name,
Datasource,
  CASE
    WHEN SRC_Dept_id IS NULL THEN TRUE
    ELSE FALSE 
  END AS is_bad_data
FROM departments

